
FOR THE JSON  DATA: 20230916_01-16-54-377_1018_userinfo.json. 

{"CGUID":"1018","Date":"20230916_01:16:54:377",

"ShotResult":[

    7.6970219612121582, 25.65673828125, -----> Head speed (m/s)  is sum of first and second column = 7.697021961212158 + 25.65673828125

    -0.608303964138031, ------> Face angle (degree)

    8.4586247339757392E-07, -----> Attack angle (degree)

    -2.7872023582458496, -----> Swing path (degree)

    2.0588235855102539, ------> Swing tempo (ratio)

    97.855049133300781,  -----> Swing backtop (degree) 

    0.34999999403953552, -----> Back swing time (second) 

    0.17000000178813934, -----> Down swing time (second)

    1.0 -----> Hand (1 = RIGHT, 2 = LEFT) 

In [56]:
import os
import pandas as pd
import json
import numpy as np

# File directories 
input_directory = r'C:\Users\James\Desktop\Raw\logRaws'
output_directory = r'C:\Users\James\Desktop\Raw\userInfo'


# Preprocess data for a single file
def preprocess_single_file(input_file_path, output_file_path):

    # Load CSV data -> filter rows where Swing Stage is not 0 -> extract swing_stage + 3 accel + 3 gyro values
    input_data = pd.read_csv(input_file_path)
    input_data = input_data[input_data.iloc[:, 3] != 0]
    input_data = input_data.iloc[:, [3, 19, 20, 21, 25, 26, 27]]
    # print(input_data.shape)
    # input_data.columns = ['swing_stage', 'accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']


    # Pad remaining values with zeros -> return np.array 
    target_shape = (999,7)
    input_values = input_data.values
    pad_rows = max(0, target_shape[0] - input_values.shape[0])
    pad_cols = max(0, target_shape[1] - input_values.shape[1])
    X = np.pad(input_values, ((0, pad_rows), (0, pad_cols)), mode='constant')
    # print(X)

    # Load JSON data -> If output is not available, skip
    with open(output_file_path, 'r') as json_file:
        try:
            output_data = json.load(json_file)
            output_data_df = pd.DataFrame(output_data['ShotResult'])
            output_data_df = pd.DataFrame.transpose(output_data_df)

            columns_to_remove = [2,3,4,5,6,7,8,9]
            output_data_df = output_data_df.drop(columns=columns_to_remove)
            # output_data_df.columns = ['swingspeed1', 'swingspeed2', 'faceangle', 'attackangle', 'swingpath', 'swingbacktop']
            # print(output_data_df.shape)
            Y = np.array(output_data_df)
            # print(Y)
        except KeyError as e:
            return None, None

    return X, Y


def preprocess_all_files():
    
    X_all_data = []
    Y_all_data = []

    # Iterate through all files
    for input_file in os.listdir(input_directory):

        # Construct the output file path based on the common prefix
        common_prefix = '_'.join(input_file.split('_')[:-1]) + '_'
        output_file = common_prefix + 'userinfo.json'
        output_file_path = os.path.join(output_directory, output_file)

        # Construct the input file path
        input_file_path = os.path.join(input_directory, input_file)

        # Preprocess the data for the current file
        X, Y = preprocess_single_file(input_file_path, output_file_path)
    
        # Append the preprocessed data to the list if both values exist
        if X is not None and Y is not None:
            X_all_data.append(X)
            Y_all_data.append(Y)

    return np.array(X_all_data), np.array(Y_all_data)


In [57]:
import tensorflow as tf

def percentage_difference_loss(y_true, y_pred):
    # Calculate percentage difference
    percentage_diff = tf.abs((y_true - y_pred) / tf.maximum(tf.abs(y_true), tf.keras.backend.epsilon()))
    
    # Take the mean across all elements in the batch
    mean_percentage_diff = tf.reduce_mean(percentage_diff) * 100
    
    return mean_percentage_diff

In [59]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
import pandas as pd

X_data, Y_data  = preprocess_all_files()

print(type(X_data))
print(X_data.shape)
print(type(Y_data))
print(Y_data.shape)

# Split data
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.2, random_state=42)

# Define model
model = Sequential()
model.add(LSTM(64, input_shape=(999, 7), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(2, activation='relu'))

# Compile modle
model.compile(optimizer='adam', loss='mae', metrics=['mae'])
model.summary()

# Train model
model.fit(X_train, Y_train, epochs=50, batch_size=64) 

# Evaluate model
score = model.evaluate(X_test, Y_test)
print(score)

tf.saved_model.save(model, r"C:\Users\James\Documents\GitHub\IMU_Golf_ML\BallFlightModelRNN")

<class 'numpy.ndarray'>
(3823, 999, 7)
<class 'numpy.ndarray'>
(3823, 1, 2)
Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_34 (LSTM)              (None, 999, 64)           18432     
                                                                 
 dropout_34 (Dropout)        (None, 999, 64)           0         
                                                                 
 lstm_35 (LSTM)              (None, 64)                33024     
                                                                 
 dropout_35 (Dropout)        (None, 64)                0         
                                                                 
 dense_17 (Dense)            (None, 2)                 130       
                                                                 
Total params: 51586 (201.51 KB)
Trainable params: 51586 (201.51 KB)
Non-trainable params: 0 (0.00 Byte)
_____

INFO:tensorflow:Assets written to: C:\Users\James\Documents\GitHub\IMU_Golf_ML\BallFlightModelRNN\assets
